In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import pandas as pd
import seaborn as sns
from panco2 import PressureProfileFitter
from utils import gNFW
import results

In [3]:
path = "../Tests/test0/"

ppf = PressureProfileFitter(
    f"{path}/map.fits",
    1,
    2,
    0.865,
    M_500=3.5e14,
)

r_bins = np.logspace(
    np.log10(ppf.cluster.arcsec2kpc(9.0)), np.log10(ppf.cluster.arcsec2kpc(180.0)), 5
)

ppf.define_model("binned", r_bins)
P_bins = gNFW(ppf.model.r_bins, *ppf.cluster.A10_params)
par_vec = np.append(P_bins, [-11.9, 0.0])
par_dic = ppf.model.par_vec2dic(par_vec)

ppf.add_filtering(beam_fwhm=18.0)
ppf.define_priors(
    P_bins=[ss.loguniform(0.1 * P, 10.0 * P) for P in P_bins],
    conv=ss.norm(-11.90, 1.19),
    zero=ss.norm(0.0, 1e-4)
)
ppf.dump_to_file(f"{path}/ppf.panco2")

# MCMC

In [ ]:
f = ppf.run_mcmc(30, 1e4, 6, n_check=1e3, out_chains_file=f"{path}/raw_chains.npz")

/Users/fkeruzore/.miniconda3/envs/panco2/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
 10%|█████████████▉                                                                                                                             | 1002/10000 [00:46<07:14, 20.69it/s]

    1000 iterations


 20%|███████████████████████████▊                                                                                                               | 2004/10000 [01:32<06:02, 22.07it/s]

    2000 iterations


 30%|█████████████████████████████████████████▋                                                                                                 | 3003/10000 [02:17<05:18, 21.96it/s]

...

 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                           | 8003/10000 [06:07<01:30, 21.96it/s]

    8000 iterations


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏             | 9003/10000 [06:53<00:44, 22.27it/s]

    9000 iterations


 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍    | 9668/10000 [07:23<00:15, 22.07it/s]

# Results

In [ ]:
chains_clean = results.load_chains(f"{path}/raw_chains.npz", 200, 20)

In [ ]:
results.mcmc_corner_plot(chains_clean, model=ppf.model, show_probs=False, filename=f"{path}/corner.png")
results.mcmc_trace_plot(chains_clean, filename=f"{path}/trace.png")

In [ ]:
plt.close("all")
results.set_plot_style("seaborn")
fig = results.plot_results(ppf, chains_clean, truth_vec=par_vec)
fig.savefig(f"{path}/results.pdf")